# 🚏 City2Graph + GNN Demo: Paris Street Network

This notebook demonstrates how **city2graph** and **Graph Neural Networks (GNNs)** can be used to analyze urban systems. 

We will:
1. Download Paris’s street network from OpenStreetMap.
2. Convert it into a graph using `city2graph`.
3. Build a **Graph Convolutional Network (GCN)** with PyTorch Geometric.
4. Train the GCN to predict whether a street node is a **major intersection**.

This is a **toy example** — but the workflow generalizes to tasks like flood exposure prediction, land value modelling, or multi-city comparisons.

In [ ]:

# !pip install city2graph osmnx geopandas networkx torch torch-geometric matplotlib

In [ ]:
import osmnx as ox
import city2graph as c2g
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

ox.settings.log_console = False  # silence osmnx logs

## 1. Download Paris Street Network

In [ ]:
city = "Paris, France"
G = ox.graph_from_place(city, network_type="drive")
nodes, edges = ox.graph_to_gdfs(G)

print(nodes.shape, edges.shape)
G

## 2. Convert to CityGraph (city2graph)

In [ ]:
cgraph = c2g.CityGraph.from_geodataframes(
    nodes, edges,
    node_id_col="osmid",
    edge_src_col="u",
    edge_dst_col="v"
)

# Convert edge index to PyTorch tensor
edge_index = torch.tensor(list(cgraph.edge_index.values.T), dtype=torch.long)

# Features: degree + coordinates
deg = np.array([d for _, d in G.degree()])
coords = np.stack([nodes.geometry.x.values, nodes.geometry.y.values], axis=1)
x = torch.tensor(np.hstack([deg.reshape(-1,1), coords]), dtype=torch.float)

# Labels: binary target (major intersection = degree > 3)
y = torch.tensor((deg > 3).astype(int), dtype=torch.long)

data = Data(x=x, edge_index=edge_index, y=y)
data

## 3. Visualize Nodes by Label (Ground Truth)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
nodes.plot(ax=ax, markersize=5, color=["red" if label==1 else "blue" for label in y.numpy()])
ax.set_title("Paris intersections: red = major (>3 roads), blue = minor")
plt.show()

## 4. Define a Simple Graph Convolutional Network

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

model = GCN(in_channels=data.x.shape[1], hidden_channels=16, out_channels=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

## 5. Train/Test Split and Training Loop

In [ ]:
n = data.num_nodes
idx = torch.randperm(n)
train_mask = idx[:int(0.8*n)]
test_mask = idx[int(0.8*n):]

for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        pred = out.argmax(dim=1)
        acc = (pred[test_mask] == data.y[test_mask]).sum() / len(test_mask)
        print(f"Epoch {epoch}, Loss {loss.item():.4f}, Test Acc {acc:.3f}")

## 6. Visualize Predictions

In [ ]:
model.eval()
pred = model(data.x, data.edge_index).argmax(dim=1).detach().numpy()

fig, ax = plt.subplots(figsize=(6,6))
nodes.plot(ax=ax, markersize=5, color=["green" if label==1 else "gray" for label in pred])
ax.set_title("Predicted major intersections (green)")
plt.show()

## ✅ Conclusion

- We built a graph representation of Paris streets with **city2graph**.
- We trained a simple **GCN** to classify intersections as major/minor.
- The pipeline is **generalizable to other cities and tasks** (flood risk, land value, adaptation scenarios).

This demo can serve as a **proof of concept** for multi-city comparative urban modelling.